# Projet Intelligence Artificielle

## Besoin client 3 : Système d'alerte pour les tempêtes

### Préparation des données

In [54]:
# Importation des librairies nécessaires

import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import pickle as pk

In [55]:
# Importation des bases de données 
data_prof = pd.read_csv('./Données/Data_Arbre.csv')
data_clean = pd.read_csv("./Données/data_clean.csv", encoding='utf-8', sep=";", decimal=",")

In [56]:
# Affichage des premières lignes des données 
# data_prof.head()


In [57]:
# Affichage des premières lignes des données de notre base de données
data_clean.head()

,X,Y,OBJECTID,created_date,created_user,src_geo,clc_quartier,clc_secteur,id_arbre,haut_tot,...,nomfrancais,GlobalID,CreationDate,Creator,EditDate,Editor,feuillage,remarquable,lon,lat
0,1.720320e+06,8.294619e+06,1,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Richelieu,24.0,0.0,...,RAS,{476EB2CE-1FD4-4F89-8162-79D75651225A},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.282540,49.850458
1,1.720898e+06,8.293531e+06,2,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,24.0,0.0,...,RAS,{B5A9F630-92C5-4B8A-A934-0CABDA46085E},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290521,49.840654
2,1.720894e+06,8.293542e+06,3,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,53.0,0.0,...,RAS,{F5914EAD-05CD-4ADF-A7C9-55EFF91B2ABE},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290460,49.840756
3,1.720902e+06,8.293545e+06,4,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,54.0,0.0,...,RAS,{41168E06-B7C0-43CD-B8FE-7495B6E93AB5},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290568,49.840783
4,1.721089e+06,8.293619e+06,5,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,63.0,0.0,...,RAS,{4F0E4B12-4612-4F61-9911-43684831FD7D},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.293178,49.841441


In [58]:
# Récupération des colonnes désirées 
data = data_clean[["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","age_estim", "fk_prec_estim","fk_pied","fk_situation"]]
data.head()

,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,age_estim,fk_prec_estim,fk_pied,fk_situation
0,0.0,0.0,0.0,SUPPRIMÉ,11.170052,7.481492,Inexistant,Alignement
1,0.0,0.0,0.0,ABATTU,11.170052,7.481492,Inexistant,Alignement
2,0.0,0.0,0.0,SUPPRIMÉ,11.170052,7.481492,Inexistant,Alignement
3,0.0,0.0,0.0,SUPPRIMÉ,11.170052,7.481492,Inexistant,Alignement
4,0.0,0.0,0.0,ABATTU,11.170052,7.481492,Inexistant,Alignement


In [59]:
# Autre méthode pour récuperer les colonnes désirées
colonnes = ["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","age_estim", "fk_prec_estim","fk_pied","fk_situation"]
data_new = data_clean[colonnes]

In [60]:
# Corrélation entre les variables numériques
correlation = data.corr(method='pearson', numeric_only= True).style.background_gradient(cmap='Pastel1')
correlation

,haut_tot,haut_tronc,tronc_diam,age_estim,fk_prec_estim
haut_tot,1.000000,0.441569,0.678304,0.678828,0.463950
haut_tronc,0.441569,1.000000,0.327135,0.505737,0.292037
tronc_diam,0.678304,0.327135,1.000000,0.857629,0.508601
age_estim,0.678828,0.505737,0.857629,1.000000,0.855497
fk_prec_estim,0.463950,0.292037,0.508601,0.855497,1.000000


In [61]:
# Retrait des lignes dont on ne se sert pas

index = data[(data["fk_arb_etat"] == 'SUPPRIMÉ') | 
             (data["fk_arb_etat"]=='ABATTU') | 
             (data["fk_arb_etat"]=='EN PLACE') | 
             (data["fk_arb_etat"]=='REMPLACÉ')].index
data.drop(index, inplace = True)
data.head()

C:\Users\mathi\AppData\Local\Temp\ipykernel_23244\1464909206.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(index, inplace = True)


,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,age_estim,fk_prec_estim,fk_pied,fk_situation
12,0.0,0.0,0.0,Essouché,11.170052,7.481492,Inexistant,Alignement
14,0.0,0.0,0.0,Essouché,11.170052,7.481492,Inexistant,Alignement
15,0.0,0.0,0.0,Essouché,11.170052,7.481492,Inexistant,Alignement
16,0.0,0.0,0.0,Essouché,11.170052,7.481492,Inexistant,Alignement
19,0.0,0.0,0.0,Non essouché,16.245164,10.000000,fosse arbre,Alignement


In [62]:
# Binarisation des valeurs de fk_arb_etat

data.loc[data["fk_arb_etat"] == "Essouché","fk_arb_etat"] = 1
data.loc[data["fk_arb_etat"] != 1,"fk_arb_etat"] = 0
data.fk_arb_etat = data.fk_arb_etat.astype(int)

C:\Users\mathi\AppData\Local\Temp\ipykernel_23244\1889407845.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fk_arb_etat = data.fk_arb_etat.astype(int)


In [63]:
# Changement des ordinalités dans les colonnes en charactère
encodeur = OrdinalEncoder()
cols = ["fk_pied","fk_situation"]
# cols = ["clc_quartier","fk_situation","feuillage"]
changement = data[cols]
data[cols] = encodeur.fit_transform(changement)

C:\Users\mathi\AppData\Local\Temp\ipykernel_23244\1348326787.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[cols] = encodeur.fit_transform(changement)


In [64]:
# Visualisation des données transformées
print(data)

      haut_tot  haut_tronc  tronc_diam  fk_arb_etat  age_estim  fk_prec_estim  \
12         0.0         0.0         0.0            1  11.170052       7.481492   
14         0.0         0.0         0.0            1  11.170052       7.481492   
15         0.0         0.0         0.0            1  11.170052       7.481492   
16         0.0         0.0         0.0            1  11.170052       7.481492   
19         0.0         0.0         0.0            0  16.245164      10.000000   
...        ...         ...         ...          ...        ...            ...   
8205       4.0         2.0        17.0            1   7.140417       2.000000   
8221       9.0         3.0        85.0            1  25.346017       5.000000   
8222       9.0         3.0        86.0            1  25.493089       5.000000   
8223       9.0         4.0        98.0            1  29.581774       5.000000   
8311       5.0         2.0        25.0            0   8.284093       2.000000   

      fk_pied  fk_situation

In [65]:
# Nouvelle corrélation entre les variables numériques
new_correlation = data.corr(method='pearson', numeric_only= True).style.background_gradient(cmap='Pastel1')
new_correlation

,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,age_estim,fk_prec_estim,fk_pied,fk_situation
haut_tot,1.000000,0.665377,0.729910,0.090665,0.699162,0.237857,0.296670,0.409936
haut_tronc,0.665377,1.000000,0.621905,0.049869,0.696205,0.209201,0.162339,0.279502
tronc_diam,0.729910,0.621905,1.000000,0.090675,0.886681,0.288290,0.224560,0.251526
fk_arb_etat,0.090665,0.049869,0.090675,1.000000,0.024848,-0.095218,0.002397,0.146225
age_estim,0.699162,0.696205,0.886681,0.024848,1.000000,0.668092,0.233696,0.140904
fk_prec_estim,0.237857,0.209201,0.288290,-0.095218,0.668092,1.000000,0.135763,-0.166544
fk_pied,0.296670,0.162339,0.224560,0.002397,0.233696,0.135763,1.000000,0.180847
fk_situation,0.409936,0.279502,0.251526,0.146225,0.140904,-0.166544,0.180847,1.000000


In [66]:
# Classification des données en X

x = data[["haut_tot","haut_tronc","tronc_diam","age_estim", "fk_prec_estim","fk_pied","fk_situation"]]
print(x)
print(x.shape)

      haut_tot  haut_tronc  tronc_diam  age_estim  fk_prec_estim  fk_pied  \
12         0.0         0.0         0.0  11.170052       7.481492      3.0   
14         0.0         0.0         0.0  11.170052       7.481492      3.0   
15         0.0         0.0         0.0  11.170052       7.481492      3.0   
16         0.0         0.0         0.0  11.170052       7.481492      3.0   
19         0.0         0.0         0.0  16.245164      10.000000      7.0   
...        ...         ...         ...        ...            ...      ...   
8205       4.0         2.0        17.0   7.140417       2.000000      5.0   
8221       9.0         3.0        85.0  25.346017       5.000000      5.0   
8222       9.0         3.0        86.0  25.493089       5.000000      5.0   
8223       9.0         4.0        98.0  29.581774       5.000000      5.0   
8311       5.0         2.0        25.0   8.284093       2.000000      5.0   

      fk_situation  
12             0.0  
14             0.0  
15          

In [67]:
# Classification des données en Y
y = data[["fk_arb_etat"]]
print(y)
print(y.shape)

      fk_arb_etat
12              1
14              1
15              1
16              1
19              0
...           ...
8205            1
8221            1
8222            1
8223            1
8311            0

[197 rows x 1 columns]
(197, 1)


In [68]:
print(np.unique(y))

[0 1]


In [69]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)

In [70]:
print(y_test)

      fk_arb_etat
7757            1
14              1
1760            1
1747            1
1294            0
1374            1
2385            0
2536            1
160             1
1241            1
2043            1
3966            1
1237            1
2173            1
1168            0
4671            0
1341            0
8311            0
1300            0
1299            0
711             1
4724            1
2952            1
1289            0
265             0
4652            1
1247            1
1270            0
2819            1
16              1
3378            0
1194            0
1017            1
8095            1
4298            1
2722            0
4300            0
2512            1
1699            1
1672            1
5594            1
5690            1
2080            1
3964            1
4829            1
2454            1
831             1
4720            1
3677            1
1307            0
4312            1
3679            1
3637            1
2111            1
1312      

### Apprentissage Supervisé pour la classification

#### Classification binaire

In [71]:
randomforest = RandomForestClassifier(n_estimators= 200, random_state= 42)
randomforest.fit(x_train,y_train)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=200, random_state=42)

In [72]:
gridsearch = GridSearchCV(estimator=randomforest, param_grid= {'n_estimators' : [i*25 for i in range(1,18)]},cv= 5, n_jobs=-1)
meilleur_model = gridsearch.fit(x_train, y_train)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [73]:
print(gridsearch.best_estimator_)

RandomForestClassifier(n_estimators=325, random_state=42)


### Métrique pour la classification

In [74]:
taux = cross_val_score(meilleur_model, x_train, y_train, cv = 3, scoring='accuracy')
print(taux)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[0.7173913  0.76086957 0.77777778]


In [75]:
print(np.mean(taux))

0.752012882447665


#### Matrice de confusion

In [76]:
predictions = cross_val_predict(gridsearch, x_train, y_train, cv = 3)
print(predictions)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0
 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1]


In [77]:
matrice = confusion_matrix(y_train,predictions, normalize = "true")
print(matrice)

[[0.45238095 0.54761905]
 [0.11578947 0.88421053]]


### Autres classifieurs

In [78]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators= 175, random_state= 42)
ada.fit(x_train,y_train)

gridsearch = GridSearchCV(estimator=ada, param_grid= {'n_estimators' : [i*25 for i in range(1,20)]},cv= 5, n_jobs=-1)
meilleur_model = gridsearch.fit(x_train, y_train)

taux = cross_val_score(meilleur_model, x_train, y_train, cv = 3, scoring='accuracy')
print(taux)

print(np.mean(taux))

predictions = cross_val_predict(gridsearch, x_train, y_train, cv = 3)
print(predictions)

matrice = confusion_matrix(y_train,predictions, normalize = "true")
print(matrice)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[0.73913043 0.63043478 0.71111111]
0.6935587761674719


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 0
 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1]
[[0.35714286 0.64285714]
 [0.15789474 0.84210526]]


In [79]:
mlp = MLPClassifier(hidden_layer_sizes=(28,60),random_state= 42)
mlp.fit(x_train,y_train)

taux = cross_val_score(mlp, x_train, y_train, cv = 3, scoring='accuracy')
print(taux)

print(np.mean(taux))

predictions = cross_val_predict(mlp, x_train, y_train, cv = 3)
print(predictions)

matrice = confusion_matrix(y_train,predictions, normalize = "true")
print(matrice)

[0.76086957 0.67391304 0.68888889]

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y


0.7078904991948471
[1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
[[0.11904762 0.88095238]
 [0.03157895 0.96842105]]


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### SMOTE

In [80]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42, n_jobs=-1)
xsmote, ysmote = smote.fit_resample(x_train,y_train)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\imblearn\over_sampling\_smote\base.py:370: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [81]:
ada_smote = AdaBoostClassifier(n_estimators= 175, random_state= 42)
ada_smote.fit(xsmote,ysmote)

gridsearch = GridSearchCV(estimator=ada_smote, param_grid= {'n_estimators' : [i*25 for i in range(1,20)]},cv= 5, n_jobs=-1)
meilleur_model = gridsearch.fit(xsmote, ysmote)

taux = cross_val_score(meilleur_model, xsmote, ysmote, cv = 3, scoring='accuracy')
print(taux)

print(np.mean(taux))

predictions = cross_val_predict(gridsearch, xsmote, ysmote, cv = 3)
print(predictions)

matrice = confusion_matrix(ysmote,predictions, normalize = "true")
print(matrice)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[0.71875    0.77777778 0.79365079]
0.7633928571428571


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[1 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1 0 1 1 1 1 1
 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1]
[[0.72631579 0.27368421]
 [0.2        0.8       ]]


Random forest

In [82]:
random = RandomForestClassifier(n_estimators= 175, random_state= 42)
random.fit(xsmote,ysmote)

gridsearch = GridSearchCV(estimator=random, param_grid= {'n_estimators' : [i*25 for i in range(3,15)]},cv= 5, n_jobs=-1)
meilleur_model = gridsearch.fit(xsmote, ysmote)

taux = cross_val_score(meilleur_model, xsmote, ysmote, cv = 3, scoring='accuracy')
print(taux)

print(np.mean(taux))

predictions = cross_val_predict(gridsearch, xsmote, ysmote, cv = 3)
print(predictions)

matrice = confusion_matrix(ysmote,predictions, normalize = "true")
print(matrice)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[0.703125   0.88888889 0.9047619 ]
0.8322585978835978


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[1 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0
 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0
 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0]
[[0.76842105 0.23157895]
 [0.10526316 0.89473684]]


base test

In [83]:
predictions = gridsearch.predict(x_test)
print(predictions)

matrice = confusion_matrix(y_test,predictions, normalize = "true")
print(matrice)

[0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1
 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1]
[[0.72222222 0.27777778]
 [0.23809524 0.76190476]]


MLP


In [84]:
mlp_sote = MLPClassifier(hidden_layer_sizes=(28,60),random_state= 42)
mlp_sote.fit(xsmote,ysmote)

taux = cross_val_score(mlp_sote, xsmote, ysmote, cv = 3, scoring='accuracy')
print(taux)

print(np.mean(taux))

predictions = cross_val_predict(mlp_sote, xsmote, ysmote, cv = 3)
print(predictions)

matrice = confusion_matrix(ysmote,predictions, normalize = "true")
print(matrice)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

[0.671875   0.63492063 0.6984127 ]
0.6684027777777777


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization 

[0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0
 0 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 0
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 1 1 0 1 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1
 0 0 0 0 0]
[[0.72631579 0.27368421]
 [0.38947368 0.61052632]]


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Préparation du Script

In [85]:
# Enregistrement du modèle 

dico = {'modele' : random, 'encodeur' : encodeur}

pk.dump(dico, open('RandomForest_Besoin_client_3.pkl','wb'))

In [86]:
# Récupération des méthodes

param = pk.load(open('RandomForest_Besoin_client_3.pkl','rb'))